## **Step 3 data insert and retrieve**

### Existing table of my database.

In [1]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.orm import relationship, declarative_base

## base class for my models
Base = declarative_base()


class Users(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    username = Column(String, nullable=False)
    age = Column(Integer)
    balance = Column(Float)
    
    def __repr__(self):
        return f"<User(id={self.id}, username='{self.username}', age={self.age}, balance={self.balance})>" 
    
    
class Address(Base):
    __tablename__ = 'addresses'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))  # Foreign key to users table
    email = Column(String, nullable=False)
    
    user = relationship("Users", backref="addresses")
    
    
    def __repr__(self):
        return f"<Address(id={self.id}, email='{self.email}', user_id={self.user_id})>"

## **DB engine**

In [2]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()
DB_URL = os.getenv("PG_DATABASE_URL")

# Connect to your local PostgreSQL database
engine = create_engine(DB_URL)

## ***STAT***

In [3]:
from sqlalchemy.orm import joinedload, selectinload
from sqlalchemy import func, text
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd


In [4]:
from sqlalchemy.orm import sessionmaker
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [5]:
# Cell 2 - Create missing tables (no-op if already created)
Base.metadata.create_all(bind=engine)
print("Tables ensured (users, addresses).")


Tables ensured (users, addresses).


## **Insert the Data**

In [6]:
session = SessionLocal()

In [7]:
# Cell 3 - Insert users and addresses and link them via relationships
users_to_add = [
    Users(username="Al Amin", age=28, balance=120.50),
    Users(username="Aminul", age=32, balance=75.00),
    Users(username="Niloy", age=22, balance=200.0),
]

addresses_to_add = [
    Address(email="alamin@example.com"),
    Address(email="anubyk@gmail.com"),
    Address(email="hello_lie@company.com"),
]



with session as db:
    try:
        # Add users first
        db.add_all(users_to_add)
        db.flush()  # assigns IDs to users in this transaction (but not committed yet)

        # Link addresses to users using object references (no manual FK needed)
        addresses_to_add[0].user = users_to_add[0]  # Alice
        addresses_to_add[1].user = users_to_add[1]  # Bob
        addresses_to_add[2].user = users_to_add[2]  # Charlie

        db.add_all(addresses_to_add)
        db.commit()
        print("Inserted users and addresses (and linked via relationships).")
    except SQLAlchemyError as e:
        db.rollback()
        print("Error during insert:", e)


Inserted users and addresses (and linked via relationships).


## **Read the data form db**

In [ ]:
with SessionLocal() as db:
    all_users = db.query(Users).order_by(Users.id).all()
    rows = []
    for u in all_users:
        rows.append(
            {
                "id": u.id,
                "username": u.username,
                "age": u.age,
                "balance": u.balance,
                "email": [a.email for a in u.addresses]
                
            }
        )
    
    display(pd.DataFrame(rows))

,id,username,age,balance,addresses
0,1,Al Amin,28,120.5,[alamin@example.com]
1,2,Aminul,32,75.0,[anubyk@gmail.com]
2,3,Niloy,22,200.0,[hello_lie@company.com]


### **The N+1 problem & eager-loading solutions (`selectinload` and `joinedload`)**

In [13]:
# Cell 5 - Demonstration: eager loading to avoid N+1
from time import perf_counter

def measure_access(with_options=None):
    with SessionLocal() as db:
        start = perf_counter()
        q = db.query(Users)
        if with_options:
            q = q.options(with_options)
        users = q.limit(10).all()
        # Access related addresses (would otherwise issue separate queries per user)
        totals = []
        for u in users:
            totals.append(len(u.addresses))
        end = perf_counter()
        return end - start, totals

t1, totals1 = measure_access(None)  # lazy load -> likely many queries
t2, totals2 = measure_access(selectinload(Users.addresses))  # uses IN + one query for addresses
t3, totals3 = measure_access(joinedload(Users.addresses))  # uses a JOIN (one query)

print("Lazy load time:", t1)
print("selectinload time:", t2)
print("joinedload time:", t3)
print("Counts (example):", totals2[:5])


Lazy load time: 0.004927499998302665
selectinload time: 0.005574700000579469
joinedload time: 0.009984399999666493
Counts (example): [1, 1, 1]


## **Query across relationships with .`join()` and `.outerjoin()`**

In [14]:
# Cell 6 - Join filters
with SessionLocal() as db:
    # Users who have a @gmail.com address
    gmail_users = db.query(Users).join(Users.addresses).filter(Address.email.ilike('%@gmail.com')).all()
    print("Gmail users:", gmail_users)

    # Users without any addresses (LEFT OUTER JOIN + filter Address.id is NULL)
    users_no_addr = db.query(Users).outerjoin(Address).filter(Address.id == None).all()
    print("Users without addresses:", users_no_addr)


Gmail users: [<User(id=2, username='Aminul', age=32, balance=75.0)>]
Users without addresses: []


In [15]:
# Cell 7 - Aggregation: address counts per user (including users with 0)
with SessionLocal() as db:
    results = (
        db.query(
            Users.id,
            Users.username,
            func.count(Address.id).label("addr_count")
        )
        .outerjoin(Address)
        .group_by(Users.id)
        .order_by(func.count(Address.id).desc())
        .all()
    )
    df = pd.DataFrame(results, columns=["id", "username", "addr_count"])
    display(df)


,id,username,addr_count
0,2,Aminul,1
1,3,Niloy,1
2,1,Al Amin,1
